In [ ]:
%%capture
%load_ext sql
%config SqlMagic.feedback = False
%config SqlMagic.displaylimit = 0
%config SqlMagic.displaycon = False

In [ ]:
import datetime

start = datetime.datetime.now() - datetime.timedelta(days=90)
end = datetime.datetime.now()

In [ ]:
%%sql report_districts <<
SELECT 
    r.district as district,
    r.primary_statute as classification,
    r.primary_complaint as complaint,
    r.location_state as state,
    COUNT(*) count
FROM cts_forms_report r
WHERE district notnull AND primary_complaint notnull
AND create_date >= '{{ start }}'
GROUP BY complaint, district, classification, state
HAVING COUNT(*) > 1
ORDER BY count desc

In [ ]:
def make_data_div(district, complaint, classification, state, count):
    district_dropdown.append(
        f'<option value="{district}">{district}</option>'
    )
    return f"""<tr class="district-wrapper {district}">
            <td><span tabindex="0">{complaint}</span></td>
            <td><span tabindex="0">{district}</span></td>
            <td><span tabindex="0">{state}</span></td>
            <td><span tabindex="0">{classification}</span></td>
            <td><a class="display-block" href="http://localhost:8000/form/view/?district={district}&primary_complaint={complaint}&primary_statute={classification}&grouping=default">
                View {count} reports
            </a></td>
            </tr>
        """

In [ ]:
from IPython.display import display, HTML
import numpy as np
primary_complaint_choices = {
    'workplace': 'Workplace discrimination or other employment-related problem',
    'housing': 'Housing discrimination or harassment',
    'education': 'Discrimination at a school, educational program or service, or related to receiving education',
    'police': 'Mistreated by police, correctional staff, or inmates',
    'voting': 'Voting rights or ability to vote affected',
    'commercial_or_public': 'Discriminated against in a commercial location or public place',
    'something_else': 'Something else happened',
}
readable_start = start.strftime("%B %d, %Y")
readable_end = end.strftime("%B %d, %Y")
report_district_data = report_districts.DataFrame()
district_dropdown = []
districts = report_district_data['district'].values
complaints = [
  primary_complaint_choices.get(complaint_choice)
  for complaint_choice
  in report_district_data['complaint'].values
]
classifications = report_district_data['classification'].values
states = report_district_data['state'].values
counts = report_district_data['count'].values
records = zip(districts, complaints, classifications, states, counts)
data_divs = [
  make_data_div(district, complaint, classification, state, count)
  for district, complaint, classification, state, count in records
]

display(HTML(f"""
    <div class="crt-portal-card light maxh-mobile-lg overflow-y-scroll">
    <h3>Repeated primary complaints by district</h3>
    <select name="incident_district" class="usa-select incident-district" aria-label="Incident district">
        <option value="">Select a district</option>
        {''.join(sorted(np.unique(district_dropdown)))}
    </select>
    <p style="font-size: 0.75rem">
      <em>Repeated report types by primary complaint and district submitted between
        <strong>{readable_start}</strong> and
        <strong>{readable_end}</strong>
      </em>
    </p>
    <br />
    <div class="intake-table">
    <table class="usa-table crt-table">
    <thead>
        <tr>
        <th>
        <span tabindex="0">Primary complaint</span>
        </th>
        <th>
        <span tabindex="0">District</span>
        </th>
        <th>
        <span tabindex="0">State</span>
        </th>
        <th>
        <span tabindex="0">Classification</span>
        </th>
        <th>
        <span tabindex="0"># of Reports</span>
            </th>
        </tr>
    </thead>
    <tbody>
    {''.join(data_divs)}
    </tbody>
    </table>
    </div>
    </div>
"""))
